In [25]:
import pickle
import os
import numpy as np
from utils import Fold, Gestures
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [26]:

def load_pickle(basepath, name):
    save_path = os.path.join(basepath, name)

    with open(save_path, 'rb') as file:
        dictionary = pickle.load(file)
        return dictionary

def display_results(data: dict):
    for model_type in data:
        print("%s results using %d iterations:" % (model_type, conv_lstm.get(model_type).get('iterations')))
        sorted_folds = list(conv_lstm.get(model_type).get('folds').keys())
        sorted_folds.sort(key=lambda x: x.value)
        for fold in sorted_folds:
            print("\t%d-fold results:" % fold.value)
            fold_results = conv_lstm.get(model_type).get('folds').get(fold)

            acc_per_fold = np.array(list(map(lambda x: x['acc_per_fold'], fold_results)))
            loss_per_fold = np.array(list(map(lambda x: x['loss_per_fold'], fold_results)))
            print("\t\tIndividual accuracies:")
            for acc in acc_per_fold:
                print("\t\t%.3f" % acc.mean(), end='')
            acc_per_fold = acc_per_fold.flatten()
            loss_per_fold = loss_per_fold.flatten()
            print("\n\t\tOverall acc: %.2f Std: %.2f" % (acc_per_fold.mean(), acc_per_fold.std()))
            print("\t\tOverall loss: %.2f" % loss_per_fold.mean())


def draw_conf_matrix(confusions, base_path: str, name: str):
    reshaped = confusions.reshape(confusions.shape[0] * confusions.shape[1], 10, 10)

    overall_confusion = np.zeros((reshaped.shape[1], reshaped.shape[2]))
    for conf in reshaped:
        overall_confusion = np.add(overall_confusion, conf)

    # Normalize predictions
    normalized_confusion = overall_confusion / overall_confusion.sum(axis=1, keepdims=1)
    normalized_confusion = normalized_confusion.round(decimals=2)

    save_path = os.path.join(base_path, f"{name.replace(' ', '_')}.svg")
    if not os.path.exists(base_path):
        os.mkdir(base_path)

    ConfusionMatrixDisplay(normalized_confusion, display_labels=Gestures).plot(cmap='Blues', xticks_rotation='vertical')
    plt.title(name)
    plt.savefig(save_path, bbox_inches="tight")

    plt.show()

In [27]:
# Analyze accuracies
conv_lstm = load_pickle('../results', 'conv_lstm.pickle')
display_results(conv_lstm)

ConvLSTM 16 results using 3 iterations:
	5-fold results:
		Individual accuracies:
		65.624		64.478		67.240
		Overall acc: 65.78 Std: 4.22
		Overall loss: 1.09
	10-fold results:
		Individual accuracies:
		67.322		67.897		66.715
		Overall acc: 67.31 Std: 5.54
		Overall loss: 1.05
ConvLSTM 32 results using 3 iterations:
	5-fold results:
		Individual accuracies:
		69.084		68.828		70.179
		Overall acc: 69.36 Std: 3.19
		Overall loss: 1.12
	10-fold results:
		Individual accuracies:
		69.533		69.305		70.132
		Overall acc: 69.66 Std: 5.06
		Overall loss: 1.06
ConvLSTM 64 results using 3 iterations:
	5-fold results:
		Individual accuracies:
		70.107		69.969		71.198
		Overall acc: 70.42 Std: 3.95
		Overall loss: 1.29
	10-fold results:
		Individual accuracies:
		70.358		69.387		70.206
		Overall acc: 69.98 Std: 5.81
		Overall loss: 1.32
ConvLSTM 128 results using 3 iterations:
	5-fold results:
		Individual accuracies:
		70.538		69.873		70.156
		Overall acc: 70.19 Std: 3.05
		Overall loss: 1.52
	10

In [28]:
# Analyze accuracies
lstm = load_pickle('../results', 'lstm.pickle')
display_results(lstm)

AttributeError: 'NoneType' object has no attribute 'get'

In [ ]:
conv_lstm = load_pickle('../results', 'conv_lstm_new.pickle')
fold_ten = conv_lstm.get('ConvLSTM 64').get('folds').get(Fold.TEN)

confusions = np.array(list(map(lambda x: x['confusion_per_fold'], fold_ten)))
draw_conf_matrix(confusions, '../results/figs/', "Confusion Matrix ConvLSTM 64-unit")

In [ ]:
# Analyze file sizes
base = '../models'
for model in os.listdir(base):
    print(model)
    path = os.path.join(base, model)
    tflite_size = os.path.getsize(os.path.join(path, f"{model}.tflite"))
    tflite_quantized_size = os.path.getsize(os.path.join(path, f"{model}_quantized.tflite"))
    print("\tNormal size: %d bytes (%.1f Kb)\n\tQuantized size: %d bytes (%.1f Kb)" % (tflite_size, round(float(tflite_size / 1000), 1), tflite_quantized_size, round(float(tflite_quantized_size / 1000), 1)))